In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests11 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = False
data_tuned_epsilon = False
epsilon = .5

size weirdness rn...

In [5]:
a = 0.25
num_params=9

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.96 & 0.83 & 0.96 & 0.96   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.10 & 0.02 & 0.01   \\
Model 2 & 0.06 & 0.06 & 0.03 & 0.07 & 0.02 & 0.03   \\
\hline
\end{tabular}
(array([0.934, 0.006, 0.06 ]), array([0.934, 0.006, 0.06 ]), array([0.962, 0.01 , 0.028]), array([0.826, 0.102, 0.072]), array([0.96 , 0.022, 0.018]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.964, 0.01 , 0.026]), -4.315094280745273, 11.00502119712782, 11.018308033059927)


# power 0.... need to see...

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.83 & 0.73 & 0.55 & 0.55 & 0.57   \\
Model 1 & 0.18 & 0.17 & 0.27 & 0.44 & 0.45 & 0.43   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.82, 0.18, 0.  ]), array([0.826, 0.174, 0.   ]), array([0.726, 0.274, 0.   ]), array([0.554, 0.44 , 0.006]), array([0.554, 0.446, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.572, 0.428, 0.   ]), 6.39716624951433, 5.180985981708911, 5.342089268648015)


In [8]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.85 & 0.85 & 0.72 & 0.50 & 0.54 & 0.56   \\
Model 1 & 0.15 & 0.15 & 0.28 & 0.49 & 0.46 & 0.44   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.846, 0.154, 0.   ]), array([0.85, 0.15, 0.  ]), array([0.72, 0.28, 0.  ]), array([0.504, 0.49 , 0.006]), array([0.536, 0.464, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.562, 0.438, 0.   ]), 6.488581819632585, 4.938415469525958, 5.307875653376323)


In [9]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.83 & 0.69 & 0.48 & 0.49 & 0.54   \\
Model 1 & 0.17 & 0.17 & 0.31 & 0.51 & 0.51 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.828, 0.172, 0.   ]), array([0.83, 0.17, 0.  ]), array([0.692, 0.308, 0.   ]), array([0.478, 0.514, 0.008]), array([0.49, 0.51, 0.  ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.536, 0.464, 0.   ]), 6.557457579732575, 4.964189619539192, 5.3335551403249095)


In [10]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.81 & 0.81 & 0.74 & 0.54 & 0.55 & 0.58   \\
Model 1 & 0.19 & 0.19 & 0.26 & 0.45 & 0.45 & 0.42   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.808, 0.192, 0.   ]), array([0.81, 0.19, 0.  ]), array([0.738, 0.262, 0.   ]), array([0.544, 0.454, 0.002]), array([0.546, 0.454, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.576, 0.424, 0.   ]), 6.508661588426715, 5.4197867573962295, 5.372737796262595)


In [11]:
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/50,nobs**(1/10) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.82 & 0.73 & 0.53 & 0.55 & 0.54   \\
Model 1 & 0.18 & 0.18 & 0.27 & 0.47 & 0.45 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.82, 0.18, 0.  ]), array([0.822, 0.178, 0.   ]), array([0.734, 0.266, 0.   ]), array([0.526, 0.468, 0.006]), array([0.552, 0.448, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.544, 0.456, 0.   ]), 6.711087054399047, 5.208298576862239, 5.361782559837809)


# a  = .25, k= 9

In [12]:
a = 0.25
num_params=9



In [13]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 0.94 & 0.82 & 0.94 & 0.96   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.12 & 0.05 & 0.02   \\
Model 2 & 0.10 & 0.09 & 0.04 & 0.06 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.9  , 0.004, 0.096]), array([0.902, 0.004, 0.094]), array([0.944, 0.016, 0.04 ]), array([0.82 , 0.124, 0.056]), array([0.936, 0.048, 0.016]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.964, 0.018, 0.018]), -3.911313351810294, 6.016293003794439, 6.061311505202296)


In [14]:
nobs = 500

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.97 & 0.83 & 0.95 & 0.96   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.10 & 0.03 & 0.02   \\
Model 2 & 0.08 & 0.08 & 0.02 & 0.07 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.916, 0.004, 0.08 ]), array([0.916, 0.004, 0.08 ]), array([0.97 , 0.008, 0.022]), array([0.83 , 0.096, 0.074]), array([0.954, 0.03 , 0.016]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.958, 0.018, 0.024]), -4.393711218117798, 8.060851747379639, 8.050789161626145)


In [15]:
nobs = 100

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.84 & 0.95 & 0.96 & 0.83 & 0.92 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.13 & 0.08 & 0.01   \\
Model 2 & 0.16 & 0.05 & 0.03 & 0.03 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.842, 0.002, 0.156]), array([0.952, 0.002, 0.046]), array([0.962, 0.01 , 0.028]), array([0.834, 0.134, 0.032]), array([0.918, 0.076, 0.006]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.986, 0.012, 0.002]), -4.358493134662849, 4.269986399102957, 4.389396412056291)


# a  = .25, k= 4, something not right?

In [16]:
a = 0.25
num_params=4



In [17]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.95 & 0.81 & 0.94 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.02 & 0.11 & 0.03 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.03 & 0.09 & 0.03 & 0.01   \\
\hline
\end{tabular}
(array([0.964, 0.004, 0.032]), array([0.964, 0.004, 0.032]), array([0.948, 0.024, 0.028]), array([0.808, 0.106, 0.086]), array([0.944, 0.03 , 0.026]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.988, 0.004, 0.008]), -1.5171603097697304, 5.391274210789007, 5.692024969284526)


# a  = .25, K = 19

In [18]:
a = 0.25
num_params=19


In [19]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.74 & 0.80 & 0.96 & 0.84 & 0.93 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.14 & 0.07 & 0.01   \\
Model 2 & 0.26 & 0.20 & 0.03 & 0.02 & 0.00 & 0.01   \\
\hline
\end{tabular}
(array([0.74, 0.  , 0.26]), array([0.8, 0. , 0.2]), array([0.958, 0.012, 0.03 ]), array([0.844, 0.136, 0.02 ]), array([0.926, 0.07 , 0.004]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.972, 0.014, 0.014]), -9.24646285762889, 6.1848653101344135, 6.708649349004275)


# evidence of power

In [20]:
nobs = 500

a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.84 & 0.56 & 0.58 & 0.62   \\
Model 1 & 0.02 & 0.02 & 0.16 & 0.43 & 0.42 & 0.38   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.98 , 0.016, 0.004]), array([0.982, 0.016, 0.002]), array([0.844, 0.156, 0.   ]), array([0.562, 0.432, 0.006]), array([0.578, 0.422, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.624, 0.376, 0.   ]), 1.7897886327619523, 5.520745143803879, 6.3442099809984045)


In [21]:
nobs = 100

a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.97 & 1.00 & 0.78 & 0.37 & 0.34 & 0.67   \\
Model 1 & 0.02 & 0.00 & 0.22 & 0.63 & 0.66 & 0.33   \\
Model 2 & 0.01 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.97 , 0.016, 0.014]), array([1., 0., 0.]), array([0.78, 0.22, 0.  ]), array([0.374, 0.626, 0.   ]), array([0.336, 0.664, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.67, 0.33, 0.  ]), 0.5496442939926551, 5.250215040100687, 5.789127563191473)


In [22]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.59 & 0.59 & 0.66 & 0.50 & 0.56 & 0.56   \\
Model 1 & 0.41 & 0.41 & 0.34 & 0.49 & 0.44 & 0.44   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.588, 0.412, 0.   ]), array([0.59, 0.41, 0.  ]), array([0.66, 0.34, 0.  ]), array([0.5  , 0.494, 0.006]), array([0.558, 0.442, 0.   ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.558, 0.442, 0.   ]), 8.86424037901693, 4.689365815333646, 4.810010329919391)


In [23]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.82 & 0.48 & 0.47 & 0.66   \\
Model 1 & 0.02 & 0.02 & 0.18 & 0.52 & 0.53 & 0.34   \\
Model 2 & 0.01 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.978, 0.016, 0.006]), array([0.982, 0.016, 0.002]), array([0.824, 0.176, 0.   ]), array([0.484, 0.516, 0.   ]), array([0.47, 0.53, 0.  ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.658, 0.342, 0.   ]), 1.1294606425584286, 5.787942320324092, 6.160829000417222)


# evidence of power 2

In [24]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.53 & 0.56 & 0.76 & 0.91 & 0.64   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.04 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.47 & 0.44 & 0.20 & 0.09 & 0.36   \\
\hline
\end{tabular}
(array([0.01, 0.  , 0.99]), array([0.534, 0.   , 0.466]), array([0.564, 0.   , 0.436]), array([0.76 , 0.038, 0.202]), array([0.91 , 0.002, 0.088]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.642, 0.   , 0.358]), -20.409770110109733, 5.499352802706429, 6.179000083724352)


In [25]:
nobs = 500


a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.02 & 0.39 & 0.70 & 0.76 & 0.92 & 0.64   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.04 & 0.00 & 0.00   \\
Model 2 & 0.98 & 0.61 & 0.30 & 0.20 & 0.08 & 0.36   \\
\hline
\end{tabular}
(array([0.02, 0.  , 0.98]), array([0.392, 0.   , 0.608]), array([0.696, 0.   , 0.304]), array([0.764, 0.038, 0.198]), array([0.916, 0.   , 0.084]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.64, 0.  , 0.36]), -20.126338924931748, 6.040678334791432, 6.241528452680817)


In [26]:
nobs = 100


a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.96 & 0.41 & 0.81 & 0.86 & 0.57   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.04 & 0.59 & 0.17 & 0.14 & 0.43   \\
\hline
\end{tabular}
(array([0.01, 0.  , 0.99]), array([0.96, 0.  , 0.04]), array([0.406, 0.   , 0.594]), array([0.812, 0.016, 0.172]), array([0.86 , 0.004, 0.136]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.566, 0.   , 0.434]), -20.752807637385647, 5.6939654098777055, 5.786517096164335)


In [27]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.19 & 0.19 & 0.61 & 0.60 & 0.67 & 0.51   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.81 & 0.81 & 0.39 & 0.40 & 0.33 & 0.49   \\
\hline
\end{tabular}
(array([0.186, 0.   , 0.814]), array([0.19, 0.  , 0.81]), array([0.606, 0.   , 0.394]), array([0.598, 0.006, 0.396]), array([0.672, 0.   , 0.328]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.506, 0.   , 0.494]), -12.83551185554732, 4.945644153371244, 5.0270403017315735)


In [28]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out,data_tuned_epsilon=data_tuned_epsilon)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.57 & 0.64 & 0.79 & 0.93 & 0.67   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.04 & 0.01 & 0.00   \\
Model 2 & 0.99 & 0.43 & 0.36 & 0.17 & 0.06 & 0.33   \\
\hline
\end{tabular}
(array([0.01, 0.  , 0.99]), array([0.572, 0.   , 0.428]), array([0.638, 0.   , 0.362]), array([0.794, 0.04 , 0.166]), array([0.934, 0.006, 0.06 ]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.67, 0.  , 0.33]), -19.936491545252256, 5.571474945834001, 6.136829190171078)
